In [0]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, sum, window, desc

# Create a SparkSession
spark = SparkSession.builder.appName("Retail Analytics").getOrCreate()
#  Load all three datasets using PySpark
products_df = spark.read.csv("/FileStore/tables/products.csv", header=True, inferSchema=True)
sales_df = spark.read.csv("/FileStore/tables/store_sales.csv", header=True, inferSchema=True)
stores_df = spark.read.csv("/FileStore/tables/stores.csv", header=True, inferSchema=True)

In [0]:
# clean the data
sales_df = sales_df.na.drop()
products_df = products_df.na.drop()
stores_df = stores_df.na.drop()

sales_df = sales_df.withColumn("sale_date", to_date(col("sale_date")))
sales_df = sales_df.withColumn("quantity", col("quantity").cast("int"))
sales_df = sales_df.withColumn("unit_price", col("unit_price").cast("double"))
sales_df = sales_df.withColumn("discount", col("discount").cast("double"))
products_df = products_df.withColumn("cost_price", col("cost_price").cast("double"))

from pyspark.sql.functions import lower, trim

products_df = products_df.withColumn("category", lower(trim(col("category"))))
stores_df = stores_df.withColumn("region", lower(trim(col("region"))))

In [0]:
# Join datasets to create a master table combining store, product, and sales data
joined_df = sales_df.join(products_df, on="product_id", how="left")
joined_df = joined_df.join(stores_df, on="store_id", how="left")
joined_df.show(5)

+--------+----------+----------+--------+----------+--------+---------+------+----------+--------------------+------+
|store_id|product_id| sale_date|quantity|unit_price|discount| category| brand|cost_price|            location|region|
+--------+----------+----------+--------+----------+--------+---------+------+----------+--------------------+------+
|       9|       133|2025-10-12|       3|    219.95|  141.63|furniture|BrandD|   1932.18|       New Ericmouth|  west|
|       3|       114|2025-09-21|       3|   5007.68|  438.38|     toys|BrandB|   3433.39|   Lake Tammychester|  east|
|       3|       110|2025-08-21|       8|   5898.02|  204.32|     toys|BrandA|   2393.44|   Lake Tammychester|  east|
|       8|       143|2025-10-05|       6|    914.68|   94.51| clothing|BrandE|    4053.2|North Kristopherb...| south|
|       9|       131|2025-09-03|       4|   4968.49|  354.57|groceries|BrandC|   2587.55|       New Ericmouth|  west|
+--------+----------+----------+--------+----------+----

In [0]:
# Add derived fields:
from pyspark.sql.functions import col, round

final_df = (
    joined_df
    .withColumn("gross_amount", round(col("quantity") * col("unit_price"), 2))
    .withColumn("net_amount", round(col("gross_amount") - col("discount"), 2))
    .withColumn("profit", round(col("net_amount") - (col("quantity") * col("cost_price")), 2))
)
final_df.show()

+--------+----------+----------+--------+----------+--------+-----------+------+----------+--------------------+------+------------+----------+---------+
|store_id|product_id| sale_date|quantity|unit_price|discount|   category| brand|cost_price|            location|region|gross_amount|net_amount|   profit|
+--------+----------+----------+--------+----------+--------+-----------+------+----------+--------------------+------+------------+----------+---------+
|       9|       133|2025-10-12|       3|    219.95|  141.63|  furniture|BrandD|   1932.18|       New Ericmouth|  west|      659.85|    518.22| -5278.32|
|       3|       114|2025-09-21|       3|   5007.68|  438.38|       toys|BrandB|   3433.39|   Lake Tammychester|  east|    15023.04|  14584.66|  4284.49|
|       3|       110|2025-08-21|       8|   5898.02|  204.32|       toys|BrandA|   2393.44|   Lake Tammychester|  east|    47184.16|  46979.84| 27832.32|
|       8|       143|2025-10-05|       6|    914.68|   94.51|   clothing|Bra

In [0]:
# Save the cleaned data as cleaned_sales.csv
final_df.write.mode("overwrite").option("header", True).csv("/dbfs/FileStore/clean_data/cleaned_sales")

In [0]:
display(final_df)

store_id,product_id,sale_date,quantity,unit_price,discount,category,brand,cost_price,location,region,gross_amount,net_amount,profit
9,133,2025-10-12,3,219.95,141.63,furniture,BrandD,1932.18,New Ericmouth,west,659.85,518.22,-5278.32
3,114,2025-09-21,3,5007.68,438.38,toys,BrandB,3433.39,Lake Tammychester,east,15023.04,14584.66,4284.49
3,110,2025-08-21,8,5898.02,204.32,toys,BrandA,2393.44,Lake Tammychester,east,47184.16,46979.84,27832.32
8,143,2025-10-05,6,914.68,94.51,clothing,BrandE,4053.2,North Kristopherborough,south,5488.08,5393.57,-18925.63
9,131,2025-09-03,4,4968.49,354.57,groceries,BrandC,2587.55,New Ericmouth,west,19873.96,19519.39,9169.19
3,115,2025-08-19,1,2327.6,394.71,groceries,BrandB,661.3,Lake Tammychester,east,2327.6,1932.89,1271.59
5,107,2025-10-04,5,3837.24,288.99,electronics,BrandC,3022.4,Lake David,north,19186.2,18897.21,3785.21
9,123,2025-09-18,7,3921.41,58.91,groceries,BrandB,3638.25,New Ericmouth,west,27449.87,27390.96,1923.21
10,136,2025-10-24,4,3556.59,3.6,furniture,BrandB,4619.93,New Melanie,south,14226.36,14222.76,-4256.96
7,124,2025-09-11,8,1664.1,327.27,electronics,BrandA,3626.44,Lake David,south,13312.8,12985.53,-16025.99
